## This notebook is an end to end build of your initial database 

Load your packages!

In [ ]:
# standard library
import os
import json
import textwrap
import warnings
from typing import List, Optional, Tuple
from pprint import pprint
from collections import Counter
from datetime import datetime, date
import re
import time
from enum import Enum

# External Libraries
import pandas as pd
from tqdm import tqdm
from pydantic import BaseModel, PydanticDeprecatedSince20, Field
from huggingface_hub.utils import _deprecation
from concurrent.futures import ThreadPoolExecutor, as_completed
from numpy import dot
from numpy.linalg import norm

# Langchain specific
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader, TextLoader
from langchain_community.vectorstores import LanceDB
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser

# LanceDB specific
import lancedb
from lancedb.pydantic import LanceModel, Vector

In [ ]:
# load functions

def Salty_Detective(root_path: str, extensions: List[str], export_csv: bool = False) -> List[Tuple[str, str, int]]:
    """
    Scans through all subfolders from root_path and identifies files
    matching user-defined extensions. Optionally exports to CSV.

    Parameters:
        root_path (str): The directory path to search.
        extensions (List[str]): File extensions to search for (e.g., ['.pdf', '.txt']).
        export_csv (bool): If True, saves results to a timestamped CSV and opens it.

    Returns:
        List of (filename, full_path, is_repeated)
    """
    extensions = [ext.lower() for ext in extensions]
    collected_files = []

    for dirpath, _, filenames in os.walk(root_path):
        for file in filenames:
            if any(file.lower().endswith(ext) for ext in extensions):
                full_path = os.path.join(dirpath, file)
                collected_files.append((file, full_path))

    name_counts = Counter([name for name, _ in collected_files])
    results = [(name, path, int(name_counts[name] > 1)) for name, path in collected_files]
    df = pd.DataFrame(results, columns=["File Name", "Path", "Does it Repeat (0 or 1)"])
    if export_csv:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"Salty_Detective_Results_{timestamp}.csv"
        df.to_csv(filename, index=False)
        print(f"\n CSV exported as: {filename}")

        try:
            os.startfile(filename)
        except Exception as e:
            print(f"There is an issue opening the file automatically: {e}")

    return df


def extract_metadata(text: str, file_path: str) -> dict:
    '''
    Function to scrape keywords from the chunks
    '''
    prompt = f"""
Given the following text, extract:

1. A list of 10–20 keywords (comma-separated)


Text:
{text}

Return in this format:
KEYWORDS: ...
"""
    response = llm.invoke([{"role": "user", "content": prompt}])
    lines = response.content.splitlines()

    keywords = []
    for line in lines:
        if line.startswith("KEYWORDS:"):
            keywords = [kw.strip() for kw in line[len("KEYWORDS:"):].split(",")]


    return {
        "key_words": keywords,
        "file_extension": file_path  # full path used as metadata
    }

def safe_embed_documents(rows, max_chars=3000):
    """
    Tries to embed all rows. If a 413 error occurs, truncates long rows and retries.
    """
    try:
        texts = [row["text"] for row in rows]
        return tei_endpoint.embed_documents(texts)

    except Exception as e:
        if "413" in str(e) or "Payload Too Large" in str(e):
            print("Payload too large. Truncating and retrying...")

            truncated_rows = []
            for row in rows:
                text = row["text"]
                if len(text) > max_chars:
                    row = row.copy()
                    row["text"] = text[:max_chars]
                truncated_rows.append(row)

            texts = [row["text"] for row in truncated_rows]
            return tei_endpoint.embed_documents(texts)

        else:
            raise  # re-raise unknown errors

def is_bad_ocr(text, symbol_threshold=0.3, short_line_ratio=0.5, gibberish_line_ratio=0.5):
    '''
    function to search for poor lines of text in a md format, adjustable threshold if OCR has a known type of failure to look for
    '''

    lines = text.splitlines()
    total_lines = len(lines)
    if total_lines == 0:
        return True

    # Count lines with mostly non-alphanumerics
    noisy_lines = sum(1 for line in lines if len(re.findall(r'[^\w\s]', line)) / (len(line) + 1e-5) > 0.5)

    # Count short lines
    short_lines = sum(1 for line in lines if len(line.strip()) <= 3)

    # Detect gibberish lines (e.g., repeated single characters)
    gibberish_lines = sum(1 for line in lines if re.match(r'^([^\w\s])\1{2,}$', line.strip()))

    # Check overall symbol frequency
    total_chars = len(text)
    symbol_count = len(re.findall(r'[^\w\s]', text))
    symbol_ratio = symbol_count / (total_chars + 1e-5)

    # Calculate line ratios
    short_line_ratio_actual = short_lines / total_lines
    gibberish_ratio_actual = gibberish_lines / total_lines

    # Determine bad OCR based on thresholds
    if (
        symbol_ratio > symbol_threshold or
        short_line_ratio_actual > short_line_ratio or
        gibberish_ratio_actual > gibberish_line_ratio
    ):
        return True

    return False

def clean_ocr_text(text):
    
    '''
    A function to clean poor OCR of excess symbols, white spacea and repeated character that don't make sense
    '''
    # Remove ASCII noise lines (e.g., lines full of ~, -, _, etc.)
    text = re.sub(r'^[~\-_\\\/\.\'\*\s]{3,}$', '', text, flags=re.MULTILINE)

    # Remove excessive whitespace
    text = re.sub(r'\s{2,}', ' ', text)
    
    # Normalize newlines (remove multiple blank lines)
    text = re.sub(r'\n{2,}', '\n', text)

    # Remove control characters and non-printable ASCII
    text = re.sub(r'[^\x20-\x7E\n]', '', text)

    # Optional: remove leading/trailing whitespace from lines
    text = "\n".join([line.strip() for line in text.splitlines()])

    return text.strip()

### Step 1: Get all your file extensions

In [3]:
# === Example Setup ===
#/home/dylan/oa4910_folder/
folder = r"/mnt/c/Users/kowal/OneDrive/Documents/NPS/Quarter_7/OA4820_Capstone_Course/" #
#extensions_to_find = ['.pdf', '.txt', '.pptx', '.docx', '.xlsx', '.csv']  # all the formats we typically use
extensions_to_find = ['.pdf']  # test with three different types

# === Run Salty Detective ===
results = Salty_Detective(folder, extensions_to_find, export_csv=False)

# === Print Results ===
results

,File Name,Path,Does it Repeat (0 or 1)
0,PORTABLE_CASTLE_Quick_Look_Final_20180601.pdf,/mnt/c/Users/kowal/OneDrive/Documents/NPS/Quar...,0
1,PRESCIENT VISION Quick Look FINAL.pdf,/mnt/c/Users/kowal/OneDrive/Documents/NPS/Quar...,0
2,PROSAIC TWAIN_Quick Look_V4.pdf,/mnt/c/Users/kowal/OneDrive/Documents/NPS/Quar...,0
3,VISTA QUAKE Quick Look Final.pdf,/mnt/c/Users/kowal/OneDrive/Documents/NPS/Quar...,0
4,(U)_ACV11_Final_Report.pdf,/mnt/c/Users/kowal/OneDrive/Documents/NPS/Quar...,0
...,...,...,...
163,Wargaming_Effectiveness_2006.pdf,/mnt/c/Users/kowal/OneDrive/Documents/NPS/Quar...,0
164,Warriors_Edge.pdf,/mnt/c/Users/kowal/OneDrive/Documents/NPS/Quar...,0
165,WG_6-13_CTOC_Report.pdf,/mnt/c/Users/kowal/OneDrive/Documents/NPS/Quar...,0
166,Wing_and_a_Prayer.pdf,/mnt/c/Users/kowal/OneDrive/Documents/NPS/Quar...,0


In [4]:
paths=results.Path.to_list()

In [5]:
paths[0]

'/mnt/c/Users/kowal/OneDrive/Documents/NPS/Quarter_7/OA4820_Capstone_Course/CapStone_PDFs_MAJ_Fritzschreck/CapStone_PDFs_MAJ_Fritzschreck/PORTABLE_CASTLE_Quick_Look_Final_20180601.pdf'

### Step 2: OCR doucments

All OCR was done via linux on HPC (Hamming) at NPS

In [ ]:
'''
#!/bin/bash
#SBATCH --job-name=docling_batch
#SBATCH --output=logs/docling_%A_%a.out
#SBATCH --error=logs/docling_%A_%a.err
#SBATCH --array=0-143        # Adjust this range to match number of PDFs
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --time=23:00:00
#SBATCH --mem=16G
 
## Activate your conda environment
source /smallwork/$USER/comp3/bin/activate
## conda activate DL
 
## Get the filename for this task
PDF=$(sed -n "$((SLURM_ARRAY_TASK_ID+1))p" pdf_list.txt)
 
 
## Run docling
docling --from pdf --to md --vlm-model smoldocling --image-export-mode referenced --output ./docling_output/ --verbose "$PDF"
'''

###  Step 3: Establish connection to your embedding model and generating LLMs

In [7]:
# Define the path to the model
endpoint_path = "gpu3"
info = os.popen(f"curl -s http://trac-malenia.ern.nps.edu:8080/{endpoint_path}/info").read()
repo_id = "/".join(json.loads(info)['model_id'].split('/')[-2:])

print(f"Connected to Genrating LLM: {repo_id}")

# Create the wrapper for the endpoint
tgi_endpoint = HuggingFaceEndpoint(
    endpoint_url=f"http://trac-malenia.ern.nps.edu:8080/{endpoint_path}",
    max_new_tokens=256,
    temperature=0.0,
    do_sample=False
)

# Patch to avoid HuggingFace deprecation warnings
class LocalChatHuggingFace(ChatHuggingFace):
    def _resolve_model_id(self):
        pass

llm = LocalChatHuggingFace(
    llm=tgi_endpoint,
    model_id=repo_id,
    verbose=True
)

# check model's max context window
print(f'Model max context window: {llm.tokenizer.model_max_length}')


# Define the path to the small model
endpoint_path2 = "gpu1"
info = os.popen(f"curl -s http://trac-malenia.ern.nps.edu:8080/{endpoint_path2}/info").read()
repo_id2 = "/".join(json.loads(info)['model_id'].split('/')[-2:])

print(f"Connected to small LLM: {repo_id2}")

# Create the wrapper for the endpoint
tgi_endpoint2 = HuggingFaceEndpoint(
    endpoint_url=f"http://trac-malenia.ern.nps.edu:8080/{endpoint_path2}",
    max_new_tokens=256,
    temperature=0.0,
    do_sample=False
)

# Patch to avoid HuggingFace deprecation warnings
class LocalChatHuggingFace(ChatHuggingFace):
    def _resolve_model_id(self):
        pass

llm2 = LocalChatHuggingFace(
    llm=tgi_endpoint2,
    model_id=repo_id2,
    verbose=True
)

# check model's max context window
print(f'Model max context window: {llm2.tokenizer.model_max_length}')


endpoint_path3 = "gpu4"
info = os.popen(f"curl -s http://trac-malenia.ern.nps.edu:8080/{endpoint_path3}/info").read()
repo_id3 = "/".join(json.loads(info)['model_id'].split('/')[-2:])

print(f"Connected to embedding LLM: {repo_id3}")
# Use same endpoint for embeddings
tei_endpoint = HuggingFaceEndpointEmbeddings(
    model=f"http://trac-malenia.ern.nps.edu:8080/{endpoint_path3}/embed"  # Adjust path as needed
)

Connected to Genrating LLM: casperhansen/llama-3.3-70b-instruct-awq
Model max context window: 131072
Connected to small LLM: Qwen/Qwen2.5-7B-Instruct
Model max context window: 131072
Connected to embedding LLM: nomic-ai/nomic-embed-text-v1.5


### Step 4 Create Database Schema and establish metadata fields

In [8]:
class ChunkMetadata(BaseModel):
    file_extension: Optional[str] = ""
    title: Optional[str] = ""
    date_of_pub: Optional[date] = None
    domain: Optional[List[str]] = []
    agency: Optional[List[str]] = []
    cocom: Optional[List[str]] = []
    country: Optional[List[str]] = []
    category: Optional[str] = ""
    purpose: Optional[str] = ""
    key_words: Optional[List[str]] = []




class TextChunkSchema(LanceModel):
    id: str
    vector: Vector(768)
    metadata: ChunkMetadata = ChunkMetadata()
    text: str

# initialize a local LanceDB database
db = lancedb.connect("./lancedb")

# initialize an empty table in the database
table = db.create_table("Capstone_test_table",
                        schema=TextChunkSchema,
                        mode='overwrite', ### Only do overwrite if you want to wipe the table clean
                        #partitioned_by=["year_of_exercise", "agency_supporting", "combatant_command", "countries", "key_words"],
                        #mode='append',  # Append to the table if it already exists
                        #partitioned_by=["year_of_exercise", "agency_supporting", "combatant_command", "countries", "key_words"],
                        )

In [9]:

class Domain(str, Enum):
    Land = "Land"
    Maritime = "Maritime"
    Air = "Air"
    Space ="Space"
    Cyberspace= "Cyberspace"
    Other= "Other" #create an out for the model if no domain match well!
class COCOM(str, Enum):
    INDOPACOM = "INDOPACOM"
    PACOM = "PACOM"
    CENTCOM = "CENTCOM"
    EUCOM= "EUCOM"
    AFRICOM = "AFRICOM"
    NORTHCOM ="NORTHCOM"
    SOUTHCOM = "SOUTHCOM"
    SPACECOM =  "SPACECOM"
    CYBERCOM = "CYBERCOM"
    SOCOM = "SOCOM"
    STRATCOM = "STRATCOM"
    TRANSCOM = "TRANSCOM"
    Other= "Other" #create an out for the model if no domain match well!
class Wargame_type(str, Enum):
    policy_decision = "policy"
    capbility_requirement= 'capability'
    conop_strategy = 'conop'
    process_org = "process"
    other= "other"
class SearchSchema(BaseModel, use_enum_values=True):
    title: str = Field(description="the title of the wargame paper")
    date_of_pub: date = Field(description="the date the paper was written, if only a year is found, use January 1st. If only a month and year is found, use the 1st of the month.",
                             example=["1999-12-01","2001-01-30","2010-04-15"])
    agency: List[str] = Field(description="the list of agencies that either sponsored or hosted the wargame")
    domain: List[Domain] = Field(description="the list of domains covered in the wargame, these are limited to Land, Maritime, Air, Space and Cyberspace. An other category is left for any wargame that does" \
    "not fit one of the previous listed domains." \
    " Land: This domain encompasses the surface of the Earth, excluding the high-water mark, and includes land-based operations." \
    " Maritime: This domain includes oceans, seas, bays, estuaries, islands, and coastal areas, along with the airspace above these areas." \
    " Air: This domain refers to the atmosphere, extending from the Earth's surface to the altitude where its effects on operations become negligible." \
    " Space: This domain includes the area above the atmosphere where the effects of atmospheric drag on airborne objects become negligible." \
    " Cyberspace: This domain encompasses the interdependent networks of information technology infrastructures, the Internet, and other data, including computer systems and embedded processors." \
    " Other: This domain is only to be selected if all other domains do not match well to the wargame.")
    country: List[str] = Field(description="This is a list of participating countries or countries that are part of the wargame")
    cocom: List[COCOM] = Field (description="the list of combatant commands associated with the wargame," \
    " these are limited to AFRICOM, CENTCOM, CYBERCOM, EUCOM, INDOPACOM, NORTHCOM, SPACECOM, SOUTHCOM, STRATCOM, and TRANSCOM. An other category is left for any wargame that does" \
    "not fit one of the previous listed combatant commands." \
    "AFRICOM includes all countries in Africa accept Egypt." \
    "CENTCOM is includes Egypt, Yemen, Oman, United Arab Emirates, Qatar, Bahrain, Saudi Arabia, " \
    "Jordan, Israel, Lebanon, Syria, Iraq, Kuwait, Iran, Afganistan, Pakistan, Turkmenistan, Tajikistan, Uzbekistan, Kyrgyzstan and Kazakhstan." \
    "CYBRECOM defends the United States, countering hostile cyber actors alongside our interagency, industry, and international partners. Thic COCOM is focused on cyberspace." \
    "EUCOM is responsible for military operations in Europe and parts of Eurasia. EUCOM includes the following countries: Albania, Andorra, Austria, Belgium, Bosnia and Herzegovina, Bulgaria" \
    "Croatia, Cyprus, Czech Republic, Denmark, Estonia, Finland, France, Germany, Greece, Hungary, Iceland, Ireland, Italy, Kosovo, Latvia, Liechtenstein, " \
    "Lithuania, Luxembourg, Malta, Moldova, Monaco, Montenegro, Netherlands, North Macedonia, Norway, Poland, Portugal, Romania, San Marino, Serbia, Slovakia, Slovenia, " \
    "Spain, Sweden, Switzerland, Ukraine, United Kingdom, Vatican City. EUCOM also includes the Artic" \
    "INDOPACOM includes 38 countries: China, Japan, Taiwan, South Korea (Republic of Korea), Norht Korea (Democratic People's Republic of Korea), Mongolia, " \
    "Brunei, Cambodia, Indonesia, Laos, Myanmar (Burma), Malaysia, Philippines, Singapore, Thailand, Timor-Leste, Vietnam, Bangladesh, Bhutan,India, Nepal, Guam," \
    "Maldives, Sri Lanka, Australia, New Zealand, Papua New Guinea, Solomon Islands, Nauru, Vanuatu, Fiji, Tonga, Tuvalu, Kiribati, Marshall Islands, Samoa, American Samoa." \
    "NORTHCOM includes Canada, Mexico, Bermuda, Bahamas and the United States." \
    "SOUTHCOM includes Antigua, barbuda, Argentina, Barbados, Belize, Bolivia, Brazil,CHile,Colombia, Costa Rica, Cuba," \
    "Dominica, Dominican Republic, Ecuador, El Salvador, Grenada, Guatemala, Guyana, Haiti, Honduras, Jamaica, " \
    "Nicaragua, Panama, Paraguay, Peru, Saint Kitts and Nevis, Saunt Lucia, Saint Vincent and Grenadines, Suriname, Trinidad and Tobago, Uruguay, " \
    "Venezuela." \
    "SPACECOM conducts operations in, from and to space to deter conflict and if necessary, defeat aggression, deliver space combat power." \
    "SOCOM develops, and employs, the world's finest SOF to conduct global special operations and activities as part of the Joint Force, in concert with" \
    " the U.S. Government Interagency, Allies, and Partners, to support persistent, networked, and distributed combatant command operations and campaigns" \
    " against state and non-state actors all to protect and advance U.S. policies and objectives." \
    "STRATCOM is responsible for Strategic Deterrence, Nuclear Operations, Nuclear Command, Control, and Communications (NC3) Enterprise Operations," \
    " Joint Electromagnetic Spectrum Operations, Global Strike and Missile Threat Assessment." \
    "TRANSCOM provides transportation services and capabilities to the other combatant commands, the military services, and defense agencies." \
    " Other: This combatant command is only to be selected if all other combatant commands do not match well to the wargame." \
    "PACOM should be written as INDOPACOM"
    )
    category: Wargame_type = Field(description= "The wargame type limited to policy, capability, conop, process or other." \
    "Policy wargames explore the implications of policy decisions in complex and uncertain environments. These games involve " \
    "participants—often experts, officials, or stakeholders—who role-play different actors to test strategies, forecast outcomes,"
    " and identify potential risks and unintended consequences." \
    "Capability wargames are structured simulations used to identify, test, and refine the capabilities that an organization—typically" \
    " a military, government agency, or large institution—needs to effectively operate in future scenarios. These wargames focus less" \
    " on specific policy decisions and more on the tools, technologies, personnel, infrastructure, and systems required to achieve known strategic objectives." \
    "Conop wargames are defined as scenario-driven event designed to explore and validate how an organization will employ its forces, capabilities, or resources to " \
    "accomplish strategic objectives under realistic and often adversarial conditions. Conop wargames test operational concepts in simulated real-world or " \
    "future environments." \
    "Process wargames are secanrio-driven exercises designed to test, evaluate, and improve the internal processes, workflows, coordination mechanisms, and " \
    "organizational structures of an entity—such as a military unit or government agency under realistic or stressful conditions." \
    "If no wargame type fits the policy, capabilty, conop, or process wargame definitions, choose other.")
    purpose: str = Field(description="Brief summary of the purpose of the wargame, no more than a paragraph long")


class SearchSchema2(BaseModel):
    keywords: List[str] = Field(description="This is a list of 10-20 keywords in the chunk.")
    country: List[str] = Field(description="This is a list of countries in the chunk. This can be an empyt list if no country is mentioned in the chunk.")
    

In [ ]:

pydantic_parser = PydanticOutputParser(pydantic_object=SearchSchema)
format_instructions = pydantic_parser.get_format_instructions()

# The Pydantic model creates the formatting instructions to be included in the prompt
# Here is the what those instructions look like
print(format_instructions)


pydantic_parser2 = PydanticOutputParser(pydantic_object=SearchSchema2)
format_instructions2 = pydantic_parser2.get_format_instructions()

# The Pydantic model creates the formatting instructions to be included in the prompt
# Here is the what those instructions look like
print(format_instructions2)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"$defs": {"COCOM": {"enum": ["INDOPACOM", "PACOM", "CENTCOM", "EUCOM", "AFRICOM", "NORTHCOM", "SOUTHCOM", "SPACECOM", "CYBERCOM", "SOCOM", "STRATCOM", "TRANSCOM", "Other"], "title": "COCOM", "type": "string"}, "Domain": {"enum": ["Land", "Maritime", "Air", "Space", "Cyberspace", "Other"], "title": "Domain", "type": "string"}, "Wargame_type": {"enum": ["policy", "capability", "conop", "process", "other"], "title": "Wargame_type", "type": "string"}}, "properties": {"title": {"description": "the title of the wargame paper", "title": "Title", "ty

In [11]:
API_prompt = PromptTemplate(
        template="""
        You are an expert wargame reviewer. Analyze the following wargame report in detail.
        Break down each major metadata parameter from the format instructions, provide a brief summary.
        
        Wargame Report: {wargame_report}
        
        {format_instructions}
        
        Analyze the wargame following the exact format specified above.
        """,
        input_variables=["wargame_report"],
        partial_variables={"format_instructions": pydantic_parser.get_format_instructions()}
    )
    
chain = API_prompt | llm | pydantic_parser

### Step 4: Iterate through every document and store in Database

In [15]:
text_splitter = CharacterTextSplitter(
    separator="\n",          # Split along line breaks
    chunk_size=1500,          # Up to 500 characters per chunk
    chunk_overlap=200        # Optional: adds continuity between chunks
)

In [19]:
# get list of paths to md files in specified directory
directory = "./final_docling_output"
doc_paths = [os.path.join(directory, filename) for filename in os.listdir(directory) if filename.endswith('.md')]

## MArcus used unstructure loader...why?
docs = [TextLoader(doc_path).load() for doc_path in doc_paths]
docs_flat_full = [item for sublist in docs for item in sublist]

In [ ]:
docs_flat=docs_flat_full #allows you to test the code without running the whole dang thing
doc_batch_size=4
full_doc_metadata={}
error_log = {} 
failed_chunks = []
# Suppress deprecation warnings
warnings.filterwarnings("ignore", category=PydanticDeprecatedSince20)
warnings.filterwarnings("ignore", category=FutureWarning, module="huggingface_hub")
for j in range(0, len(docs_flat), doc_batch_size):
    remove_doc=[]
    cutoff=j+doc_batch_size-1
    if cutoff>len(docs_flat):
        cutoff=len(docs_flat)-1
    print(f'Working documents: {j} to {cutoff}')
    current_docs=docs_flat[j:j+doc_batch_size]
    full_doc_metadata={}
    for doc in reversed(current_docs): 
        print(f'Starting on {doc.metadata['source']} document')
        try:
            json_output=chain.invoke({"wargame_report":doc.page_content})  
            full_doc_metadata[doc.metadata['source']]=json_output.model_dump()
        except Exception as e:
            if "422" in str(e):
                print(f"too many tokens, attempting to shorten document with {len(doc.page_content)} characters")
                time.sleep(5)
                try:
                    if is_bad_ocr(doc.page_content):
                        print("detected bad ocr, cleaning and retrying API call")
                        text=clean_ocr_text(doc.page_content)
                        text=doc.page_content[:150000] +"..."                        
                        json_output=chain.invoke({"wargame_report":text})  
                        full_doc_metadata[doc.metadata['source']]=json_output.model_dump() 
                    else:
                        print("good ocr, document is just too long, trimming end") 
                        text=clean_ocr_text(doc.page_content)
                        text=doc.page_content[:150000] +"..."
                        json_output=chain.invoke({"wargame_report":text})  
                        full_doc_metadata[doc.metadata['source']]=json_output.model_dump() 
                except Exception as f:
                    print(f'error at {doc.metadata['source']}')
                    print(f'error code: {f}')
                    error_log[doc.metadata['source']]= str(f)
                    current_docs.remove(doc)
            elif "Read timed out." in str(e):
                print("time out error, pausing for 10 seconds")
                time.sleep(10)
                try:
                    json_output=chain.invoke({"wargame_report":doc.page_content})  
                    full_doc_metadata[doc.metadata['source']]=json_output.model_dump() 
                except Exception as f:
                    print("time out error not fixed")
                    print(f'error at {doc.metadata['source']}')
                    print(f'error code: {f}')
                    error_log[doc.metadata['source']]= str(f)
                    current_docs.remove(doc)          
           # if str(e)
            else:
                print(f'error at {doc.metadata['source']}')
                print(f'error code: {e}')
                error_log[doc.metadata['source']]= str(e)
                current_docs.remove(doc)

    # Create IDs for each chunk so we're not inserting duplicates
    existing_ids = set()
    print()
    try:
        existing_ids = set(row["id"] for row in table.to_arrow().to_pylist())
        print(f"Loaded {len(existing_ids)} existing IDs from LanceDB.")
    except Exception as e:
        print(f"Warning: Could not load existing IDs. Starting fresh. Reason: {e}")

    # Initialize counters
    skipped = 0
    processed = 0


    # Batch size for LLM + TEI
    batch_size = 32
    updates = []
    texts_to_embed = []
    rows_to_embed = []

    #remove docs where API call failed

    # Apply splitter to your loaded docs
    chunked_docs = []
    for doc in current_docs:
        chunks = text_splitter.split_text(doc.page_content)
        for i, chunk in enumerate(chunks):
            chunked_docs.append({
                "text": chunk,
                "file_path": doc.metadata.get("source", ""),
                "chunk_index": i
            })

    print(f"Created {len(chunked_docs)} text chunks.")
    # Processing loop
    for i in tqdm(range(0, len(chunked_docs), batch_size), desc="Processing chunks"):
        batch = chunked_docs[i:i + batch_size]

        # Filter out already-inserted chunks
        filtered_batch = []
        for chunk in batch:
            row_id = f"{chunk['file_path']}#{chunk['chunk_index']}"
            if row_id not in existing_ids:
                filtered_batch.append((row_id, chunk))
            else:
                skipped += 1

        if not filtered_batch:
            continue

        # Parallelize metadata extraction
        # Create worker threads
        with ThreadPoolExecutor(max_workers=5) as executor:
            # Submit tasks to the executor
            futures = {
                executor.submit(extract_metadata, chunk["text"], chunk["file_path"]): (row_id, chunk)
                for row_id, chunk in filtered_batch
            }
            # Process results as they complete
            for future in as_completed(futures):
                try:
                    metadata = future.result()
                    row_id, chunk = futures[future]
                    # Add the remaining schema-required fields
                    # ** unpack the metadata dict
                    row_metadata = ChunkMetadata(
                        **{
                            **metadata,
                            **full_doc_metadata[chunk["file_path"]]
                        }
                    ).model_dump()
                    # Package into full LanceDB-compatible row
                    rows_to_embed.append({
                        "id": row_id,
                        "text": chunk["text"],
                        "metadata": row_metadata
                    })
                    texts_to_embed.append(chunk["text"])
                    processed += 1

                except Exception as e:
                    file_path = futures[future][1]['file_path']
                    print(f"Metadata extraction failed for {file_path}: {e}")
                    failed_chunks.append(file_path)

        # Run TEI Embedding
        if texts_to_embed:
            # Embed the texts in batches
            vectors = safe_embed_documents(rows_to_embed)

            # Add vectors to the rows
            # zip the vectors with the rows of text
            for row, vector in zip(rows_to_embed, vectors):
                updates.append({
                    "id": row["id"],
                    "vector": vector,
                    "text": row["text"],
                    "metadata": row["metadata"]
                })

            existing_ids.update(row["id"] for row in rows_to_embed)
            rows_to_embed = []
            texts_to_embed = []

        # Add LanceDB updates in batches
        # Check if the updates list has reached the batch size
        if len(updates) >= 256:
            table.add(updates)
            updates = []

    # Only for the last batch which may not be full
    if texts_to_embed:
        vectors = safe_embed_documents(rows_to_embed)

        for row, vector in zip(rows_to_embed, vectors):
            updates.append({
                "id": row["id"],
                "vector": vector,
                "text": row["text"],
                "metadata": row["metadata"]
            })
        existing_ids.update(row["id"] for row in rows_to_embed)
    # final catch for updates
    if updates:
        table.add(updates)

    # Final Report
    print("\nFinished processing.")
    print(f"Chunks inserted      : {processed}")
    print(f"Chunks skipped       : {skipped}")
    print(f"Chunks failed (LLM)  : {len(failed_chunks)}")
    print(f"Total chunked_docs   : {len(chunked_docs)}")

    if failed_chunks:
        print("Failed chunks:")
        for path in failed_chunks[:5]:
            print(f" - {path}")
        if len(failed_chunks) > 5:
            print(f" ...and {len(failed_chunks) - 5} more.")

    print("\n \n")     
 

Working documents: 0 to 3
Starting on ./final_docling_output/(U)_DON_Climate_Action_2030_TTX_III_Final_Report.md document
Starting on ./final_docling_output/(U)_China_Future_report_AWC_15_May_15.md document
Starting on ./final_docling_output/(U)_ACV11_QuickLook_Final.md document
Starting on ./final_docling_output/(U)_ACV11_Final_Report.md document

Loaded 0 existing IDs from LanceDB.
Created 170 text chunks.


Processing chunks: 100%|██████████| 6/6 [02:39<00:00, 26.54s/it]



Finished processing.
Chunks inserted      : 170
Chunks skipped       : 0
Chunks failed (LLM)  : 0
Total chunked_docs   : 170

 

Working documents: 4 to 7
Starting on ./final_docling_output/(U)_EW09_Quicklook.md document
Starting on ./final_docling_output/(U)_EW09_Final_Report.md document
Starting on ./final_docling_output/(U)_EW08_Quicklook.md document
Starting on ./final_docling_output/(U)_EW08_Final_Report.md document

Loaded 170 existing IDs from LanceDB.
Created 174 text chunks.


Processing chunks: 100%|██████████| 6/6 [02:44<00:00, 27.46s/it]



Finished processing.
Chunks inserted      : 174
Chunks skipped       : 0
Chunks failed (LLM)  : 0
Total chunked_docs   : 174

 

Working documents: 8 to 11
Starting on ./final_docling_output/(U)_Syria_Analysis_Game_Final_Report.md document
Starting on ./final_docling_output/(U)_Expeditionary_Medicine_2015_Final_Report.md document
Starting on ./final_docling_output/(U)_EW11_Final_Report.md document
Starting on ./final_docling_output/(U)_EW10_Final_Report.md document


...


Finished processing.
Chunks inserted      : 1177
Chunks skipped       : 0
Chunks failed (LLM)  : 14
Total chunked_docs   : 1180
Failed chunks:
 - ./final_docling_output/QUICK.md
 - ./final_docling_output/SCYLLA_III-73_Quick_Look.md
 - ./final_docling_output/WAGCAP_Phase_II_AppB_Part_II.md
 - ./final_docling_output/WAGCAP_Phase_II_AppB_Part_II.md
 - ./final_docling_output/WAGCAP_Volume_VII_DIVWAG.md
 ...and 9 more.

 

Working documents: 160 to 161
Starting on ./final_docling_output/wmd_Wargaming_1994.md document
too many tokens, attempting to shorten document with 894102 characters
detected bad ocr, cleaning and retrying API call
Starting on ./final_docling_output/Wing_and_a_Prayer.md document
error at ./final_docling_output/Wing_and_a_Prayer.md
error code: Failed to parse SearchSchema from completion {"title": "A Wing and a Prayer", "date_of_pub": "2009-04-01", "agency": ["USDA-APHIS", "CNA"], "domain": ["Land"], "country": ["United States"], "cocom": ["USDA-APHIS"], "category": "pro

Created a chunk of size 1632, which is longer than the specified 1500
Created a chunk of size 1566, which is longer than the specified 1500
Created a chunk of size 1777, which is longer than the specified 1500
Created a chunk of size 1649, which is longer than the specified 1500
Created a chunk of size 1681, which is longer than the specified 1500
Created a chunk of size 1569, which is longer than the specified 1500
Created a chunk of size 1712, which is longer than the specified 1500
Created a chunk of size 1523, which is longer than the specified 1500
Created a chunk of size 1542, which is longer than the specified 1500
Created a chunk of size 1555, which is longer than the specified 1500
Created a chunk of size 1845, which is longer than the specified 1500
Created a chunk of size 1809, which is longer than the specified 1500
Created a chunk of size 2061, which is longer than the specified 1500
Created a chunk of size 1965, which is longer than the specified 1500
Created a chunk of s

Loaded 17601 existing IDs from LanceDB.
Created 772 text chunks.


Processing chunks: 100%|██████████| 25/25 [11:50<00:00, 28.40s/it]



Finished processing.
Chunks inserted      : 772
Chunks skipped       : 0
Chunks failed (LLM)  : 14
Total chunked_docs   : 772
Failed chunks:
 - ./final_docling_output/QUICK.md
 - ./final_docling_output/SCYLLA_III-73_Quick_Look.md
 - ./final_docling_output/WAGCAP_Phase_II_AppB_Part_II.md
 - ./final_docling_output/WAGCAP_Phase_II_AppB_Part_II.md
 - ./final_docling_output/WAGCAP_Volume_VII_DIVWAG.md
 ...and 9 more.

 



In [29]:
failed_chunks

['./final_docling_output/QUICK.md',
 './final_docling_output/SCYLLA_III-73_Quick_Look.md',
 './final_docling_output/WAGCAP_Phase_II_AppB_Part_II.md',
 './final_docling_output/WAGCAP_Phase_II_AppB_Part_II.md',
 './final_docling_output/WAGCAP_Volume_VII_DIVWAG.md',
 './final_docling_output/WAGCAP_Volume_VI_DIVWAG_Data_Req.md',
 './final_docling_output/WAGCAP_Volume_VI_DIVWAG_Data_Req.md',
 './final_docling_output/WAGCAP_Volume_VI_DIVWAG_Data_Req.md',
 './final_docling_output/WAGCAP_Volume_V_Part_III_DIVWAG_Prog_Manual.md',
 './final_docling_output/WAGCAP_Volume_V_Part_III_DIVWAG_Prog_Manual.md',
 './final_docling_output/WAGCAP_Volume_V_Part_II_DIVWAG_Prog_Manual.md',
 './final_docling_output/WAGCAP_Volume_V_Part_I_DIVWAG_Prog_Manual.md',
 './final_docling_output/WAGCAP_Volume_V_Part_I_DIVWAG_Prog_Manual.md',
 './final_docling_output/WAGCAP_Volume_V_Part_I_DIVWAG_Prog_Manual.md']

In [ ]:
failed_paths = [
    "./final_docling_output/CAMMS_v_CPX.md",
    "./final_docling_output/Counter-Insurgency_Study.md",
    "./final_docling_output/Commercial_Satellite_Report.md",
    "./final_docling_output/ExerciseViking22_Final_Report.md",
    "./final_docling_output/Olympiad_I-62.md",
    "./final_docling_output/ODNA_Transformation_Strategy_GameIV.md",
    "./final_docling_output/paper_370.md",
    "./final_docling_output/POLITICA_and_AGILE-COIN_1966.md",
    "./final_docling_output/QUICK.md",
    "./final_docling_output/WAGCAP_Volume_VII_Testing_Report.md",
    "./final_docling_output/WAGCAP_Volume_VI_DIVWAG_Data_Req.md",
    "./final_docling_output/WAGCAP_Volume_V_Part_III_DIVWAG_Prog_Manual.md",
    "./final_docling_output/WAGCAP_Volume_V_Part_II_DIVWAG_Prog_Manual.md",
    "./final_docling_output/SCYLLA_III-73_Quick_Look.md",
    "./final_docling_output/WAGCAP_Phase_II_AppB_Part_II.md",
    "./final_docling_output/WAGCAP_Volume_VII_DIVWAG.md",
    "./final_docling_output/Warriors_Edge.md",
    "./final_docling_output/WAGCAP_Volume_V_Part_I_DIVWAG_Prog_Manual.md",
    "./final_docling_output/Wing_and_a_Prayer.md",
    "./final_docling_output/WAGCAP_Phase_II_AppC.md",
    "./final_docling_output/WAGCAP_Volume_I.md",
    "./final_docling_output/Stress_Testing_Financial_Crisis_2007.md",
    "./final_docling_output/Theater_Battle_Model_Volume_VII.md",
    "./final_docling_output/WAGCAP_Phase_II.md"
]
# get list of paths to md files in specified directory
directory = "./final_docling_output"
doc_paths = [os.path.join(directory, filename) for filename in os.listdir(directory) if filename.endswith('.md')]

# Compare lists
missing_from_master = [path for path in failed_paths if path not in doc_paths]

# Output results
if missing_from_master:
    print("The following failed paths are NOT in the master list:")
    for path in missing_from_master:
        print(f" - {path}")
else:
    print("All failed paths are present in the master list.")





All failed paths are present in the master list.


In [31]:
docs = [TextLoader(doc_path).load() for doc_path in failed_paths]
docs_flat_partial = [item for sublist in docs for item in sublist]
len(docs_flat_partial)

24

In [ ]:
#### Run again against failed paths

docs_flat=docs_flat_partial #allows you to test the code without running the whole dang thing
doc_batch_size=4
full_doc_metadata={}
error_log = {} 
failed_chunks = []
# Suppress deprecation warnings
warnings.filterwarnings("ignore", category=PydanticDeprecatedSince20)
warnings.filterwarnings("ignore", category=FutureWarning, module="huggingface_hub")
for j in range(0, len(docs_flat), doc_batch_size):
    remove_doc=[]
    cutoff=j+doc_batch_size-1
    if cutoff>len(docs_flat):
        cutoff=len(docs_flat)-1
    print(f'Working documents: {j} to {cutoff}')
    current_docs=docs_flat[j:j+doc_batch_size]
    full_doc_metadata={}
    for doc in reversed(current_docs): 
        print(f'Starting on {doc.metadata['source']} document')
        try:
            json_output=chain.invoke({"wargame_report":doc.page_content})  
            full_doc_metadata[doc.metadata['source']]=json_output.model_dump()
        except Exception as e:
            if "422" in str(e):
                print(f"too many tokens, attempting to shorten document with {len(doc.page_content)} characters")
                time.sleep(5)
                try:
                    if is_bad_ocr(doc.page_content):
                        print("detected bad ocr, cleaning and retrying API call")
                        text=clean_ocr_text(doc.page_content)
                        text=doc.page_content[:150000] +"..."                        
                        json_output=chain.invoke({"wargame_report":text})  
                        full_doc_metadata[doc.metadata['source']]=json_output.model_dump() 
                    else:
                        print("good ocr, document is just too long, trimming end") 
                        text=clean_ocr_text(doc.page_content)
                        text=doc.page_content[:150000] +"..."
                        json_output=chain.invoke({"wargame_report":text})  
                        full_doc_metadata[doc.metadata['source']]=json_output.model_dump() 
                except Exception as f:
                    print(f'error at {doc.metadata['source']}')
                    print(f'error code: {f}')
                    error_log[doc.metadata['source']]= str(f)
                    current_docs.remove(doc)
            elif "Read timed out." in str(e):
                print("time out error, pausing for 10 seconds")
                time.sleep(10)
                try:
                    json_output=chain.invoke({"wargame_report":doc.page_content})  
                    full_doc_metadata[doc.metadata['source']]=json_output.model_dump() 
                except Exception as f:
                    print("time out error not fixed")
                    print(f'error at {doc.metadata['source']}')
                    print(f'error code: {f}')
                    error_log[doc.metadata['source']]= str(f)
                    current_docs.remove(doc)          
           # if str(e)
            else:
                print(f'error at {doc.metadata['source']}')
                print(f'error code: {e}')
                error_log[doc.metadata['source']]= str(e)
                current_docs.remove(doc)

    # Create IDs for each chunk so we're not inserting duplicates
    existing_ids = set()
    print()
    try:
        existing_ids = set(row["id"] for row in table.to_arrow().to_pylist())
        print(f"Loaded {len(existing_ids)} existing IDs from LanceDB.")
    except Exception as e:
        print(f"Warning: Could not load existing IDs. Starting fresh. Reason: {e}")

    # Initialize counters
    skipped = 0
    processed = 0


    # Batch size for LLM + TEI
    batch_size = 32
    updates = []
    texts_to_embed = []
    rows_to_embed = []

    #remove docs where API call failed

    # Apply splitter to your loaded docs
    chunked_docs = []
    for doc in current_docs:
        chunks = text_splitter.split_text(doc.page_content)
        for i, chunk in enumerate(chunks):
            chunked_docs.append({
                "text": chunk,
                "file_path": doc.metadata.get("source", ""),
                "chunk_index": i
            })

    print(f"Created {len(chunked_docs)} text chunks.")
    # Processing loop
    for i in tqdm(range(0, len(chunked_docs), batch_size), desc="Processing chunks"):
        batch = chunked_docs[i:i + batch_size]

        # Filter out already-inserted chunks
        filtered_batch = []
        for chunk in batch:
            row_id = f"{chunk['file_path']}#{chunk['chunk_index']}"
            if row_id not in existing_ids:
                filtered_batch.append((row_id, chunk))
            else:
                skipped += 1

        if not filtered_batch:
            continue

        # Parallelize metadata extraction
        # Create worker threads
        with ThreadPoolExecutor(max_workers=5) as executor:
            # Submit tasks to the executor
            futures = {
                executor.submit(extract_metadata, chunk["text"], chunk["file_path"]): (row_id, chunk)
                for row_id, chunk in filtered_batch
            }
            # Process results as they complete
            for future in as_completed(futures):
                try:
                    metadata = future.result()
                    row_id, chunk = futures[future]
                    # Add the remaining schema-required fields
                    # ** unpack the metadata dict
                    row_metadata = ChunkMetadata(
                        **{
                            **metadata,
                            **full_doc_metadata[chunk["file_path"]]
                        }
                    ).model_dump()
                    # Package into full LanceDB-compatible row
                    rows_to_embed.append({
                        "id": row_id,
                        "text": chunk["text"],
                        "metadata": row_metadata
                    })
                    texts_to_embed.append(chunk["text"])
                    processed += 1

                except Exception as e:
                    file_path = futures[future][1]['file_path']
                    print(f"Metadata extraction failed for {file_path}: {e}")
                    failed_chunks.append(file_path)

        # Run TEI Embedding
        if texts_to_embed:
            # Embed the texts in batches
            vectors = safe_embed_documents(rows_to_embed)

            # Add vectors to the rows
            # zip the vectors with the rows of text
            for row, vector in zip(rows_to_embed, vectors):
                updates.append({
                    "id": row["id"],
                    "vector": vector,
                    "text": row["text"],
                    "metadata": row["metadata"]
                })

            existing_ids.update(row["id"] for row in rows_to_embed)
            rows_to_embed = []
            texts_to_embed = []

        # Add LanceDB updates in batches
        # Check if the updates list has reached the batch size
        if len(updates) >= 256:
            table.add(updates)
            updates = []

    # Only for the last batch which may not be full
    if texts_to_embed:
        vectors = safe_embed_documents(rows_to_embed)

        for row, vector in zip(rows_to_embed, vectors):
            updates.append({
                "id": row["id"],
                "vector": vector,
                "text": row["text"],
                "metadata": row["metadata"]
            })
        existing_ids.update(row["id"] for row in rows_to_embed)
    # final catch for updates
    if updates:
        table.add(updates)

    # Final Report
    print("\nFinished processing.")
    print(f"Chunks inserted      : {processed}")
    print(f"Chunks skipped       : {skipped}")
    print(f"Chunks failed (LLM)  : {len(failed_chunks)}")
    print(f"Total chunked_docs   : {len(chunked_docs)}")

    if failed_chunks:
        print("Failed chunks:")
        for path in failed_chunks[:5]:
            print(f" - {path}")
        if len(failed_chunks) > 5:
            print(f" ...and {len(failed_chunks) - 5} more.")

    print("\n \n") 

Working documents: 0 to 3
Starting on ./final_docling_output/ExerciseViking22_Final_Report.md document
time out error, pausing for 10 seconds
time out error not fixed
error at ./final_docling_output/ExerciseViking22_Final_Report.md
error code: (ReadTimeoutError("HTTPConnectionPool(host='trac-malenia.ern.nps.edu', port=8080): Read timed out. (read timeout=120)"), '(Request ID: 90214190-c011-43fb-b271-b05c88cb83e8)')
Starting on ./final_docling_output/Commercial_Satellite_Report.md document
time out error, pausing for 10 seconds
time out error not fixed
error at ./final_docling_output/Commercial_Satellite_Report.md
error code: (ReadTimeoutError("HTTPConnectionPool(host='trac-malenia.ern.nps.edu', port=8080): Read timed out. (read timeout=120)"), '(Request ID: 5030e6a6-5ed9-46f2-a1e4-79df980b50c4)')
Starting on ./final_docling_output/Counter-Insurgency_Study.md document
time out error, pausing for 10 seconds
time out error not fixed
error at ./final_docling_output/Counter-Insurgency_Study

Processing chunks: 0it [00:00, ?it/s]


Finished processing.
Chunks inserted      : 0
Chunks skipped       : 0
Chunks failed (LLM)  : 0
Total chunked_docs   : 0

 

Working documents: 4 to 7
Starting on ./final_docling_output/POLITICA_and_AGILE-COIN_1966.md document


time out error, pausing for 10 seconds
time out error not fixed
error at ./final_docling_output/POLITICA_and_AGILE-COIN_1966.md
error code: (ReadTimeoutError("HTTPConnectionPool(host='trac-malenia.ern.nps.edu', port=8080): Read timed out. (read timeout=120)"), '(Request ID: a8a29e8b-04d5-4ca8-b67c-1bbeed51bedf)')
Starting on ./final_docling_output/paper_370.md document
Starting on ./final_docling_output/ODNA_Transformation_Strategy_GameIV.md document
time out error, pausing for 10 seconds
time out error not fixed
error at ./final_docling_output/ODNA_Transformation_Strategy_GameIV.md
error code: (ReadTimeoutError("HTTPConnectionPool(host='trac-malenia.ern.nps.edu', port=8080): Read timed out. (read timeout=120)"), '(Request ID: fd25ef52-2231-4634-ac52-9b283ac8ae0e)')
Starting on ./final_docling_output/Olympiad_I-62.md document
time out error, pausing for 10 seconds
time out error not fixed
error at ./final_docling_output/Olympiad_I-62.md
error code: (ReadTimeoutError("HTTPConnectionPool

Processing chunks: 100%|██████████| 2/2 [00:47<00:00, 23.61s/it]


...


Finished processing.
Chunks inserted      : 129
Chunks skipped       : 789
Chunks failed (LLM)  : 0
Total chunked_docs   : 918

 

Working documents: 20 to 23
Starting on ./final_docling_output/WAGCAP_Phase_II.md document
error at ./final_docling_output/WAGCAP_Phase_II.md
error code: Failed to parse SearchSchema from completion {"title": "Improvement of the War Gaming Capability, Phase II (WAGCAP II)", "date_of_pub": "1973-06-01", "agency": ["US Army Combat Developments Command"], "domain": ["Land"], "country": ["United States"], "cocom": ["USACDC"], "category": "capability", "purpose": "The purpose of this report is to describe the work performed and results achieved by Computer Sciences Corporation's Combat Developments Research Office in response to USACDC Work Directive 6-72, Improvement of the War Gaming Capability, Phase II."}. Got: 1 validation error for SearchSchema
cocom.0
  Input should be 'INDOPACOM', 'PACOM', 'CENTCOM', 'EUCOM', 'AFRICOM', 'NORTHCOM', 'SOUTHCOM', 'SPACECOM

Created a chunk of size 1582, which is longer than the specified 1500
Created a chunk of size 1560, which is longer than the specified 1500


Loaded 18560 existing IDs from LanceDB.
Created 101 text chunks.


Processing chunks: 100%|██████████| 4/4 [01:45<00:00, 26.48s/it]


Finished processing.
Chunks inserted      : 101
Chunks skipped       : 0
Chunks failed (LLM)  : 0
Total chunked_docs   : 101

 



In [38]:
list(error_log.keys())

['./final_docling_output/ExerciseViking22_Final_Report.md',
 './final_docling_output/Commercial_Satellite_Report.md',
 './final_docling_output/Counter-Insurgency_Study.md',
 './final_docling_output/CAMMS_v_CPX.md',
 './final_docling_output/POLITICA_and_AGILE-COIN_1966.md',
 './final_docling_output/ODNA_Transformation_Strategy_GameIV.md',
 './final_docling_output/Olympiad_I-62.md',
 './final_docling_output/WAGCAP_Volume_V_Part_III_DIVWAG_Prog_Manual.md',
 './final_docling_output/WAGCAP_Volume_VII_Testing_Report.md',
 './final_docling_output/WAGCAP_Phase_II_AppB_Part_II.md',
 './final_docling_output/WAGCAP_Phase_II_AppC.md',
 './final_docling_output/Warriors_Edge.md',
 './final_docling_output/WAGCAP_Phase_II.md',
 './final_docling_output/Theater_Battle_Model_Volume_VII.md',
 './final_docling_output/Stress_Testing_Financial_Crisis_2007.md']